## Imports

In [11]:
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from scipy.linalg import null_space
from scipy.signal import convolve2d
from skimage.measure import compare_ssim as ssim
# plt.rcParams['figure.figsize'] = (4.0, 4.0)
plt.rcParams['image.cmap'] = 'gray'

## Get the video file

In [2]:
def get_video(name_file):
    cap= cv2.VideoCapture(name_file)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        frames.append(frame)
    return frames

In [9]:
def rotate_video(video):
    out = []
    for frame in video:
        out.append(cv2.transpose(frame))
    return np.array(out)

In [15]:
def convert_to_gray_scale(video):
    out = []
    for frame in video:
        out.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.int16))
    return out

In [17]:
def convert_to_edges(video, L_th, H_th):
    out = []
    for frame in video:
        out.append(cv2.Canny(frame, L_th, H_th))
    return out

In [15]:
def convert_black_white(video):
    out = []
    for frame in video:
        frame[frame < 80] = 0
        frame[frame >= 80] = 255
        out.append(frame)
    return out

In [16]:
def normalize(lst):
    arr = np.array(lst)
    return arr / arr.max()

## Lucas-Kanade Basic OF

In [10]:
def Deriv_Gauss_x(sigma, mask_size):
    return np.array([[-i/(2*np.pi*sigma**4)*np.exp(-(i**2 + j**2)/(2*sigma**2)) for j in range(-mask_size, mask_size+1)] for i in range(-mask_size, mask_size+1)])

def Deriv_Gauss_y(sigma, mask_size):
    return np.array([[-j/(2*np.pi*sigma**4)*np.exp(-(i**2 + j**2)/(2*sigma**2)) for j in range(-mask_size, mask_size+1)] for i in range(-mask_size, mask_size+1)])

def Grad_x(img, G_dx):
    return convolve2d(img, G_dx, mode='same')

def Grad_y(img, G_dy):
    return convolve2d(img, G_dy, mode='same')

def gauss_kernel_2d(sigma, mask_size):
    return np.array([[1/(2 * np.pi * sigma ** 2) * np.exp(- (i**2 + j**2)/(2 * sigma ** 2)) for j in range(-mask_size, mask_size+1)] for i in range(-mask_size, mask_size+1)])

In [3]:
def basic_LK_OF(video, nf1, nf2, sigma_S, sigma_R, patch_size, scale_percent=100):
    im1_s = np.ones((300, 300)) * 255
    im1_s[50:100, 50:100] = 0 #frame[50:100, 50:100]
    im2_s = np.ones((300, 300)) * 255
    im2_s[55:105, 55:105] = 0 #frame[50:100, 50:100]
    im1_s = video[nf1].astype(np.float32)
    im2_s = video[nf2].astype(np.float32)
    img1_orig = im1_s#frames[nf1]
    img2_orig = im2_s#frames[nf2]
    frames = [im1_s, im2_s]
    #First smooth in temporal domain:
#     if nf1 > 1 and nf2 > 1 and len(frames) > nf1 + 2 and len(frames) > nf2 + 2:
#         img1 = np.average(frames[nf1-2:nf1+3], axis=0, weights=[1,4,6,4,1])
#         img2 = np.average(frames[nf2-2:nf2+3], axis=0, weights=[1,4,6,4,1])
#     else:
#         img1 = frames[nf1]
#         img2 = frames[nf2]
    img1 = im1_s#frames[nf1] 
    img2 = im2_s#frames[nf2]
    #resize:
    width = int(img1.shape[1] * scale_percent / 100)
    height = int(img1.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    img1 = cv2.resize(img1, dim, interpolation = cv2.INTER_LINEAR)
    img2 = cv2.resize(img2, dim, interpolation = cv2.INTER_LINEAR)
    img1_orig = cv2.resize(img1_orig, dim, interpolation = cv2.INTER_LINEAR)
    img2_orig = cv2.resize(img2_orig, dim, interpolation = cv2.INTER_LINEAR)
    mask_size = int(3 * sigma_S)
    G_dx = Deriv_Gauss_x(sigma_S, mask_size)
    G_dy = Deriv_Gauss_y(sigma_S, mask_size)
    Ix = Grad_x(img1, G_dx)
#     plt.imshow(Ix)
#     plt.show();
    Iy = Grad_y(img1, G_dy)
#     plt.imshow(Iy)
#     plt.show();
    It = img2 - img1
    gauss_kernel = gauss_kernel_2d(sigma_R, int(3 * sigma_R))
#     gauss_kernel=np.array([[1,2,1],[2,4,2],[1,2,1]])/16
#     print(gauss_kernel)
    C = [convolve2d(Ix * Ix, gauss_kernel), convolve2d(gauss_kernel, Ix * Iy), convolve2d(gauss_kernel, Iy * Iy)]
    U = np.zeros((img1.shape[0], img1.shape[1]))
    V = np.zeros((img1.shape[0], img1.shape[1]))
    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            c = np.array([[C[0][i][j], C[1][i][j]], [C[1][i][j], C[2][i][j]]])
            if np.linalg.matrix_rank(c) == 2:
                min_x = max(0, i - int(patch_size / 2))
                max_x = min(img1.shape[0], i + int(patch_size / 2) + 1)
                min_y = max(0, j - int(patch_size / 2))
                max_y = min(img1.shape[1], j + int(patch_size / 2) + 1)
                A = np.column_stack((Ix[min_x: max_x, min_y: max_y].reshape(-1, 1), Iy[min_x: max_x, min_y: max_y].reshape(-1, 1)))
                b = -It[min_x: max_x, min_y: max_y].reshape(-1, 1)
                u, v = np.matmul(np.linalg.pinv(A), b)
                U[i][j] = float(u)
                V[i][j] = float(v)
#                 print(i)
    return U

In [9]:
def OF_plot_results(U,im1, skip, arrow_size):
    x = np.arange(0, im1.shape[1], skip)
    y = np.arange(0, im1.shape[0], skip)
    X, Y = np.meshgrid(x, y)
    V = np.zeros(im1.shape)
    
    U_reduced = U[: : skip, : : skip]
    V_reduced = V[: : skip, : : skip]
#     print('X shape: ', X.shape)
#     print('U_reduced shape: ', U_reduced.shape)
    plt.figure(figsize = (30,30))
    fig, ax = plt.subplots(figsize = (30,30))
    ax.imshow(im1, cmap = 'gray')
    ax.quiver( X, Y, U_reduced, -V_reduced, width = arrow_size, scale = 70, color = 'blue')
    ax.set_aspect('equal')
    plt.title('OF_plot_results', fontsize = 20)
    plt.show()
    
    return 

## Canny edge detection

In [16]:
def Grad_o(Ix, Iy):
    return np.arctan(Iy / Ix)

def Grad_m(Ix, Iy):
    return np.sqrt(Ix ** 2 + Iy ** 2)

In [14]:
def thinning(G_magnitute, G_orientation):
    G_orientation[(G_orientation <= np.pi / 8) | (G_orientation > 7 * np.pi / 8)] = 0 # horizontal
    G_orientation[(G_orientation > np.pi / 8) & (G_orientation <= 3 * np.pi / 8)] = 1 # diagonal 45
    G_orientation[(G_orientation > 3 * np.pi / 8) & (G_orientation <= 5 * np.pi / 8)] = 2 # vertical
    G_orientation[(G_orientation > 5 * np.pi / 8) & (G_orientation <= 7 * np.pi / 8)] = 3 # diagonal 135
    thin_img = np.zeros((G_magnitute.shape))
    for i in range(1, G_magnitute.shape[0] - 1):
        for j in range(1, G_magnitute.shape[1] - 1):
            if G_orientation[i][j] == 0:
                l = G_magnitute[i][j - 1]
                r = G_magnitute[i][j + 1]
            if G_orientation[i][j] == 1:
                l = G_magnitute[i - 1][j + 1]
                r = G_magnitute[i + 1][j - 1] 
            if G_orientation[i][j] == 2:
                l = G_magnitute[i + 1][j]
                r = G_magnitute[i - 1][j] 
            if G_orientation[i][j] == 3:
                l = G_magnitute[i - 1][j - 1]
                r = G_magnitute[i + 1][j + 1]
            if G_magnitute[i][j] >= l and G_magnitute[i][j] >= r:
                thin_img[i][j] = G_magnitute[i][j]
    return thin_img


def canny(img, sigma, L_th, H_th):
    mask_size = 8  #you should compute the mask size
    G_dx = Deriv_Gauss_x(sigma, mask_size)
    G_dy = Deriv_Gauss_y(sigma, mask_size)
        
    Ix = Grad_x(img, G_dx)
    Iy = Grad_y(img, G_dy)
    
    G_orientation = Grad_o(Ix, Iy)
    G_magnitute = Grad_m(Ix, Iy)
    
    Et = thinning(G_magnitute, G_orientation)
    thin_img = Et
    thin_img = thin_img / thin_img.max()
    high_threshold = H_th
    low_threshold = L_th
    
    thin_img_high = thin_img.copy()
    thin_img_high[thin_img_high <= high_threshold] = 0
    thin_img_high[thin_img_high > high_threshold] = 1

    thin_img_low = thin_img.copy()
    thin_img_low[(thin_img_low < low_threshold) | (thin_img_low > high_threshold)] = 0
    thin_img_low[(thin_img_low <= high_threshold) & (thin_img_low >= low_threshold)] = 2

    sum_original = thin_img_high + thin_img_low
    sum_original[sum_original > 0] = 1
    kernel = np.ones((15, 15),np.uint8)
    dilate_img_high = cv2.dilate(thin_img_high, kernel, iterations = 1)
    dilate_and_low = dilate_img_high + thin_img_low
    no_dilate_img = np.uint8(dilate_and_low * sum_original)

    num_labels, labels  = cv2.connectedComponents(no_dilate_img, connectivity=8)
    no_dilate_img[no_dilate_img > 0] = 1

    unique_labels = np.unique(labels)
    for i in unique_labels:
        idx = np.where(labels == i)
        if (thin_img[idx] >= high_threshold).any():
            labels[idx] = 1
        else:
            labels[idx] = 0
    return labels

## Tracking

In [7]:
# mask_2 = np.ones((14,14))
# mask_2[:, 0] = -1
# mask_2[:, 6] = -1
# mask_2[:, 13] = -1
# mask_2[0, :] = -1
# mask_2[13, :] = -1
# mask_2[6:8, :] = -1
# plt.imshow(mask_2)

In [13]:
def track(img, kernel): #img in color, kernel grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.int16)
    window_size = kernel.shape[0]
    window = np.ones((window_size, window_size))
    shift = int(window_size / 2)
    max_corr = 0
    x_idx = 0
    for i in range(shift, img.shape[0] - shift):
        for j in range(shift, img.shape[1] - shift):
#             print('window.shape: ', window.shape)
#             print('img[i - shift : i + shift + 1, j - shift : j + shift + 1]: ', img[i - shift : i + shift + 1, j - shift : j + shift + 1])
            v = window * img[i - shift : i + shift + 1, j - shift : j + shift + 1]
#             print('v: ', v)
            if np.linalg.norm(kernel) == 0 or np.linalg.norm(v) == 0:
                continue
#             print('kernel.shape: ', kernel.shape)
#             print('v.shape: ', v.shape)
            corr = np.dot(kernel.flatten(), v.flatten()) / (np.linalg.norm(kernel) * np.linalg.norm(v))
            if corr > max_corr:
#                 print(j)
                max_corr = corr
                x_idx = j
    return x_idx

In [10]:
def crop(video, x_lim, y_lim):
    out = []
    for frame in video:
        croped = frame[y_lim[0]:y_lim[1], x_lim[0]:x_lim[1]]
        out.append(croped)
    return out

In [62]:
def frequency_calc(pos_arr, fps = 30):
    count = 0
    t = len(pos_arr)
    if len(pos_arr) < 2:
        return 0
    if pos_arr[0] > pos_arr[1]:
        count += 1
    for i in range(0, len(pos_arr) - 1):
        if pos_arr[i - 1] == pos_arr[i]:
#             print(i)
            t -= 1
        if pos_arr[i] > pos_arr[i + 1] and pos_arr[i] > pos_arr[i - 1]:
            count += 1
    return count / (len(pos_arr) / fps)

In [1]:
def intensity_window(video, window):
    s = []
    for i in range(len(video)):
        s.append(video[i][window[0]:window[1], window[2]:window[3]].sum())
    return s

In [ ]:
def intensity_count(video, window, similarity_factor):
    count = 0
    t = len(video)
    th = video[0][window[0]:window[1], window[2]:window[3]].sum()
    for i in range(len(video) - 1):
#         print(frame[window[0]:window[1], window[2]:window[3]].sum())
        present = video[i]
        next_frame = video[i + 1]
        if ssim(video[i], video[i + 1]) > similarity_factor:
            t -= 1
            continue
            
        s = video[i][window[0]:window[1], window[2]:window[3]].sum()
#         if s > 0.99 * th or s < 1.01 * th:
#             count += 1
        if s == th:
            count += 1
    return (count / (t / 30))

In [8]:
def track_video(video, kernel):
    out = []
    for i in range(len(video)):
#         print(i)
        out.append(track(video[i], kernel))
    return out

In [9]:
def close_far(video):
    out = []
    for frame in video:
        out.append(frame.sum())
    return out

In [17]:
def intensity_pixel(video, pixel):
    s = []
    for i in range(len(video)):
        s.append(video[i][pixel[0], pixel[1]])
    return s

In [12]:
def create_video_track(video, pos):
    out = []
    radius = 50
    color = (255, 0, 0)
    thickness = 4
    lineType = 2
    f = 0.0
    for i in range(len(video)):
        if i % 3 == 0 and i > 20:
            t = frequency_calc(pos[i - 20 : i + 20])
            f = round(((f + t) / 2), 2) # average to reduce outliers
#         print(f)
        video[i] = cv2.putText(video[i],f'{f}Hz', (50,600), cv2.FONT_HERSHEY_SIMPLEX , 3, (255, 0, 0), 3)
        center_coordinates = (int(pos[i] + 200), 340) # +200 because of the crop
        out.append(cv2.circle(video[i], center_coordinates, radius, color, thickness))
    return out

In [70]:
def save_video(video, name):
    writer = cv2.VideoWriter(name + '.mp4', cv2.VideoWriter_fourcc(*"DIVX"), 30,(1080, 1920))
    for i in range(len(video)):
        writer.write(video[i])
    writer.release()

In [13]:
def plot_func(pos, video, file_name):
    plt.figure(figsize = (15,8))
    plt.plot(pos, 'b')
    plt.xticks(np.arange(0, len(video), 50))
    plt.xticks(fontsize = 20)
    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.yticks(fontsize = 20)
    plt.savefig(f'/Users/galblecher/Desktop/cv_project/plots_5/{file_name}.png')
    return 

In [72]:
def paste(main, pic):
    main = Image.fromarray(np.uint8(main)).convert('RGB')
    pic = Image.fromarray(np.uint8(pic)).convert('RGB')
    main.paste(pic, (0, 1350))
    pasted = np.array(main)
    return pasted

In [80]:
def create_plots(pos, video):
    for i in range(len(pos)):
        plot_func(pos[:i], video, i)

In [75]:
def plot_on_img(video, path):
    out = []
    for i in range(len(video)): 
        plot = cv2.imread(path + f'/{i}' + '.png')
        main = video[i]
        out.append(paste(main, plot))
    return out

In [10]:
def save_results(lst, name):
    with open(name + ".txt", "w") as f:
        for l in lst:
            f.write(str(l) +"\n")

In [14]:
def create_video_track_rectangle(video, pos):
    out = []
    start_point = (100, 400) 
    end_point = (550, 750)
    color = (255, 0, 0)
    thickness = 4
    lineType = 2
    f = 0.0
    for i in range(len(video)):
        if i % 3 == 0 and i > 20:
            t = frequency_calc(pos[i - 20 : i + 20])
            f = round(((f + t) / 2), 2) # average to reduce outliers
#         print(f)
        video[i] = cv2.putText(video[i],f'{f}Hz', (50,300), cv2.FONT_HERSHEY_SIMPLEX , 3, (255, 0, 0), 3)
        out.append(cv2.rectangle(video[i], start_point, end_point, color, thickness))
    return out

In [10]:
def create_video_track_arrow(video, pos):
    out = []
    start_point = (645, 100) 
    end_point = (753, 200)
    color = (255, 0, 0)
    thickness = 4
    lineType = 2
    f = 0.0
    for i in range(len(video)):
        if i % 3 == 0 and i > 20:
            t = frequency_calc(pos[i - 20 : i + 20])
            f = round(((f + t) / 2), 2) # average to reduce outliers
#         print(f)
        video[i] = cv2.putText(video[i],f'{f}Hz', (50,600), cv2.FONT_HERSHEY_SIMPLEX , 3, (255, 0, 0), 3)
        video[i] = cv2.putText(video[i],f'The pixel ', (450, 100), cv2.FONT_HERSHEY_SIMPLEX , 1.5, (255, 0, 0), 3)
        out.append(cv2.arrowedLine(video[i], start_point, end_point, color, thickness))
    return out

In [15]:
def create_video_from_frame(frame):
    video = []
    for i in range(60):
        video.append(frame)
    return video